In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_classifier import GPTClassifier
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [3]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
        "matbench_glass",
        # "matbench_expt_is_metal",
    ],
)

Fetching matbench_expt_gap.json.gz from https://ml.materialsproject.org/projects/matbench_expt_gap.json.gz to /Users/kevinmaikjablonka/miniconda3/envs/gptchem/lib/python3.9/site-packages/matminer/datasets/matbench_expt_gap.json.gz


Fetching https://ml.materialsproject.org/projects/matbench_expt_gap.json.gz in MB: 0.038911999999999995MB [00:00, 32.59MB/s]   

Fetching matbench_steels.json.gz from https://ml.materialsproject.org/projects/matbench_steels.json.gz to /Users/kevinmaikjablonka/miniconda3/envs/gptchem/lib/python3.9/site-packages/matminer/datasets/matbench_steels.json.gz



Fetching https://ml.materialsproject.org/projects/matbench_steels.json.gz in MB: 0.010239999999999999MB [00:00, 15.24MB/s]     

2023-01-27 14:38:01 INFO     Initialized benchmark 'matbench_v0.1' with 2 tasks: 
['matbench_expt_gap', 'matbench_steels']


In [4]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [5]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    classifier = GPTClassifier(
        "glass", Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    classifier.fit(train_inputs, train_outputs.astype(int))

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = classifier.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)

In [34]:
for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        train_test_fold(task, fold)

    print(f"{task.dataset_name}: Accuracy score {task.scores['accuracy']['mean']}")
    print(f"{task.dataset_name}: ROC score {task.scores['rocauc']['mean']}")

2023-01-20 19:31:38 INFO     Dataset matbench_glass already loaded; not reloading dataset.
Skipping fold 0 of matbench_glass
Skipping fold 1 of matbench_glass
Skipping fold 2 of matbench_glass


Upload progress: 100%|██████████| 501k/501k [00:00<00:00, 662Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230120_193138/train.jsonl: file-6JWBGhgIJmZvixWvfMLCXqcL


2023-01-20 19:31:41.826 | DEBUG    | gptchem.tuner:tune:186 - Requested fine tuning. {
  "created_at": 1674239501,
  "events": [
    {
      "created_at": 1674239501,
      "level": "info",
      "message": "Created fine-tune: ft-o6TihuXk1GBiMzGF67htjut5",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-o6TihuXk1GBiMzGF67htjut5",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 500764,
      "created_at": 1674239501,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230120_193138/train.jsonl",
      "id": "file-6JWBGhgIJmZvixWvfMLCXqcL",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
   

2023-01-21 01:14:00 INFO     Recorded fold matbench_glass-3 successfully.


Upload progress: 100%|██████████| 501k/501k [00:00<00:00, 917Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230121_011400/train.jsonl: file-7GmjHOUYCkNh59lBgWjw51sJ


2023-01-21 01:14:03.474 | DEBUG    | gptchem.tuner:tune:186 - Requested fine tuning. {
  "created_at": 1674260045,
  "events": [
    {
      "created_at": 1674260045,
      "level": "info",
      "message": "Created fine-tune: ft-8KqCIWxDRnQvrjIALZv5Pndy",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-8KqCIWxDRnQvrjIALZv5Pndy",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 500890,
      "created_at": 1674260044,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230121_011400/train.jsonl",
      "id": "file-7GmjHOUYCkNh59lBgWjw51sJ",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
   

2023-01-21 01:56:48 INFO     Recorded fold matbench_glass-4 successfully.
matbench_glass: Accuracy score 0.8237676056338028
matbench_glass: ROC score 0.7746002870024067


In [37]:
task.scores

{'accuracy': {'mean': 0.8237676056338028,
  'max': 0.8433098591549296,
  'min': 0.8098591549295775,
  'std': 0.01105656230148422},
 'balanced_accuracy': {'mean': 0.7746002870024067,
  'max': 0.7951962878008911,
  'min': 0.7635506943424368,
  'std': 0.011276450569150618},
 'f1': {'mean': 0.87781787031763,
  'max': 0.8918590522478737,
  'min': 0.8671586715867159,
  'std': 0.008156367097310315},
 'rocauc': {'mean': 0.7746002870024067,
  'max': 0.7951962878008911,
  'min': 0.7635506943424368,
  'std': 0.011276450569150618},
 '_ipython_canary_method_should_not_exist_': {}}

In [36]:
from fastcore.xtras import save_pickle

In [38]:
save_pickle("glass.pkl", task.scores)

In [16]:
list(mb.tasks)[0].scores

ValueError: Cannot score unless all folds are recorded!; folds [0, 1, 2, 3, 4] not recorded!

In [39]:
save_pickle("mb.pkl", mb)

In [1]:
mb.to_file("gpt_bench.json.gz")

NameError: name 'mb' is not defined